In [20]:
!pip install mysql-connector-python

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [239]:
import mysql.connector

con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')

In [193]:
pd_bom = pd.read_sql_query('select * from bom_movie_gross', con)

In [172]:
pd_bom.to_csv('csv/filename.csv')

In [194]:
pd_bom = pd_bom.dropna()
pd_bom.dtypes


title              object
studio             object
domestic_gross    float64
foreign_gross     float64
year                int64
dtype: object

In [174]:
pd_bom = pd_bom.astype({"domestic_gross":'int64', "foreign_gross":'int64'}) 

In [175]:
pd_bom.head(10)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000,652000000,2010
1,Alice in Wonderland (2010),BV,334200000,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000,664300000,2010
3,Inception,WB,292600000,535700000,2010
4,Shrek Forever After,P/DW,238700000,513900000,2010
5,The Twilight Saga: Eclipse,Sum.,300500000,398000000,2010
6,Iron Man 2,Par.,312400000,311500000,2010
7,Tangled,BV,200800000,391000000,2010
8,Despicable Me,Uni.,251500000,291600000,2010
9,How to Train Your Dragon,P/DW,217600000,277300000,2010


In [240]:
pd_roi = pd.read_sql_query('select * from tn_movie_budgets', con)
pd_roi.to_csv('csv/roi.csv')
pd_roi.dtypes

id                    int64
release_date         object
movie                object
production_budget    object
domestic_gross       object
worldwide_gross      object
dtype: object

In [241]:
pd_roi ["release_date"] = pd.to_datetime(pd_roi ["release_date"]) 
pd_roi.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [242]:
#Cleaning those 3 columns
pd_roi ["production_budget"] = pd_roi ["production_budget"].map(lambda x: x.lstrip('$')).str.replace(',','')
pd_roi ["domestic_gross"] = pd_roi ["domestic_gross"].map(lambda x: x.strip('$,')).str.replace(',','')
pd_roi ["worldwide_gross"] = pd_roi ["worldwide_gross"].map(lambda x: x.strip('$,')).str.replace(',','')
pd_roi = pd_roi.astype({"production_budget":'int64', "domestic_gross":'int64',"worldwide_gross":'int64'}) 
pd_roi.head()


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


In [243]:
#New column roi metrics that show you 
pd_roi ["roi"] = (pd_roi ["worldwide_gross"])/pd_roi ["production_budget"]
pd_roi = pd_roi.sort_values("roi",ascending = 0)
pd_roi.dtypes

id                            int64
release_date         datetime64[ns]
movie                        object
production_budget             int64
domestic_gross                int64
worldwide_gross               int64
roi                         float64
dtype: object

In [244]:
# Selecting movies after 2000
pd_roi['year'] = pd_roi['release_date'].apply(lambda x: x.year)
pd_roi = pd_roi.loc [pd_roi['year'] > 2010]
pd_roi = pd_roi.loc [pd_roi['roi'] > 5]
pd_roi.info

<bound method DataFrame.info of       id release_date                   movie  production_budget  \
5679  80   2015-07-10             The Gallows             100000   
5211  12   2012-01-06        The Devil Inside            1000000   
5062  63   2011-04-01               Insidious            1500000   
5213  14   2015-04-17              Unfriended            1000000   
4249  50   2017-01-20                   Split            5000000   
...   ..          ...                     ...                ...   
2035  36   2017-07-21              Girls Trip           28000000   
98    99   2017-07-07  Spider-Man: Homecoming          175000000   
1883  84   2017-11-22            Darkest Hour           30000000   
1773  74   2012-07-27      Step Up Revolution           33000000   
1631  32   2013-02-08          Identity Thief           35000000   

      domestic_gross  worldwide_gross         roi  year  
5679        22764410         41656474  416.564740  2015  
5211        53262945        1017594